### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [104]:
import heapq

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [105]:
baby_df

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [106]:
#a)

#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

baby_df['review'] = baby_df['review'].apply(lambda text: remove_punctuation(str(text)))

baby_df["review"][6]

'Lovely book its bound tightly so you may not be able to add alot of photoscards aside from the designated spaces in the book Shop around before you purchase as it is currently listed at Barnes  Noble for 2995'

In [107]:
#b)
baby_df["review"] = baby_df['review'].replace(['nan'], [""])
#short test:
print(baby_df["review"][38] == baby_df["review"][38])
baby_df["review"][38]

True


''

In [108]:
#c)
baby_df = baby_df.drop(baby_df[baby_df["rating"] == 3].index)
#short test:
sum(baby_df["rating"] == 3)

0

In [109]:
#d) 
ones = baby_df.loc[baby_df["rating"] >= 4]
minus_one = baby_df.loc[baby_df["rating"] <= 2]

baby_df.loc[ones.index, 'rating'] = 1
baby_df.loc[minus_one.index, 'rating'] = -1
# baby_df["rating"] = baby_df['rating'].replace([[1, 2]], [-1])
# baby_df["rating"] = baby_df['rating'].replace([[4, 5]], [1])
#short test:
sum(baby_df["rating"]**2 != 1)
# 
# baby_df


0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [110]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names_out())
print(X_train_example.todense())



['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [111]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [112]:
#a)
X_train, X_test, y_train, y_test = train_test_split(baby_df['review'], baby_df['rating'], test_size=0.3, random_state=42)

X_train

13547     the smell was not that good that I think it wi...
134926    The audio and video quality are pretty good Ni...
139951    This play yard is perfect for when I care for ...
141273    Easy to hold onto for little ones just learnin...
114707    Odd this product seems to be high quality exce...
                                ...                        
131916                                                     
114140     Love how comfortable it is Never get back ach...
145114    After much research we decided this was the be...
161586    This is a good staple when cloth diapering and...
134183    I absolutely love my new Stork Craft crib Easy...
Name: review, Length: 116726, dtype: object

In [113]:
#b)
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train.astype('U'))
X_test = vectorizer.fit_transform(X_test.astype('U'))

X_train
    

<116726x111574 sparse matrix of type '<class 'numpy.int64'>'
	with 6219169 stored elements in Compressed Sparse Row format>

In [114]:
print(X_train[0])

  (0, 97409)	1
  (0, 89139)	1
  (0, 107087)	1
  (0, 66297)	3
  (0, 97279)	2
  (0, 43754)	2
  (0, 98428)	1
  (0, 52490)	2
  (0, 109178)	1
  (0, 14115)	1
  (0, 105877)	1
  (0, 14370)	1
  (0, 19149)	1
  (0, 8844)	1


## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [115]:
#a)
model = LogisticRegression(max_iter=100, verbose=1, n_jobs=4)

model.fit(X_train, y_train)


C:\Anaconda\envs\ds\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Anaconda\envs\ds\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    8.6s finished


LogisticRegression(n_jobs=4, verbose=1)

In [116]:
y_train

13547    -1
134926    1
139951    1
141273    1
114707   -1
         ..
131916    1
114140    1
145114   -1
161586    1
134183    1
Name: rating, Length: 116726, dtype: int64

In [141]:
#b)
print(model.coef_)
top_positive_coef = np.argpartition(model.coef_[0], -10)[-10:]
top_positive_coef
# model.coef_[0][model.coef_[0] == top_positive_coef]
# print(vectorizer.get_feature_names_out()[model.coef_])

#hint: model.coef_, vectorizer.get_feature_names()

[[ 0.00116184  0.00524893  0.0193877  ...  0.00204842  0.0004644
  -0.00050545]]


array([ 84039,  24814,  24828,  58118,  36738,   9004,  72218, 110259,
        56461,  11926], dtype=int64)

## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [118]:
#a)


In [119]:
#b)

#hint: model.predict_proba()

In [120]:
#c) 

#hint: use the results of b)

In [121]:
#d) 


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [122]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [123]:
#a)


In [124]:
#b)


In [125]:
#c)

#hint: %time, %timeit